In [84]:
import json
import numpy as np
import matplotlib.pyplot as plt
import itertools

In [62]:
with open("sondes.json") as f:
    sondes = json.load(f)
    
with open("malo.json") as f:
    malo = json.load(f)

In [63]:
KEY = "c2si-oral"

In [64]:
sondes[KEY] = np.array(sondes[KEY])
malo[KEY] = np.array(malo[KEY]) * 100

In [65]:
sondes[KEY]

In [66]:
malo[KEY]

In [70]:
difference = malo[KEY] - sondes[KEY]
maximum = np.maximum(malo[KEY], sondes[KEY])
difference

In [91]:
who_better = difference.copy()
mine = []
hers = []
for i, j in itertools.product(range(maximum.shape[0]), range(maximum.shape[1])):
    diff = round(difference[i, j], 1)
    if i == j:
        if diff == 0.0:
            who_better[i, j] = 0
        elif diff > 0.0:
            who_better[i, j] = 10
            mine += [diff]
        else:
            who_better[i, j] = -10
            hers += [diff]
    else:
        if diff == 0.0:
            who_better[i, j] = 0
        elif diff > 0.0:
            who_better[i, j] = -10
            hers += [diff]
        else:
            who_better[i, j] = 10
            mine += [diff]

In [75]:
def linear_mapping_function(x1, y1, x2, y2):
    # Calculate the slope and intercept of the linear equation
    slope = (y2 - y1) / (x2 - x1)
    intercept = y1 - slope * x1

    # Define the linear mapping lambda function
    return lambda x: slope * x + intercept


def get_sign_text(value, x, y) -> str:
    value = round(value, 1)
    if value == 0.0:
        return "="
    
    if x == y:
        if value > 0.0:
            return "+"
        return "-"
    else:
        if value > 0.0:
            return "-   "
        return "+"

In [90]:
fig, ax = plt.subplots(dpi=125)
fig.set_size_inches(15, 12, forward=True)

rect = plt.Rectangle((-0.5, -0.5), maximum.shape[1], maximum.shape[0], linewidth=2, edgecolor='black', facecolor='none')
ax.add_patch(rect)

print(np.min(difference), np.max(difference))
im = ax.imshow(who_better, interpolation='nearest', cmap=plt.cm.PiYG, vmin=-30, vmax=30)

# labels = ["p", "t", "k", "b", "d", "g", "f", "s", "ʃ", "v", "z", "ʒ"]
labels = ["a", "Ê", "Û", "Ô", "u", "y", "i", "ã", "ɔ̃", "µ", "n", "m"]

tick_marks = np.arange(len(labels))
ax.set_xticks(tick_marks)
ax.set_yticks(tick_marks)

fontsize = linear_mapping_function(1, 27, 31, 14)(len(labels))
ax.set_xticklabels([f"/{a}/" for a in labels], fontsize=fontsize, weight='bold')
ax.set_yticklabels([f"/{a}/" for a in labels], fontsize=fontsize, weight='bold')

ax.grid(False)

fontsize = linear_mapping_function(1, 23, 31, 10)(len(labels))
for i, j in itertools.product(range(maximum.shape[0]), range(maximum.shape[1])):
    color = "black"  #"white" if maximum[i, j] > 60.0 else "black"
    ax.text(j, i, round(np.abs(difference[i, j]), 1), ha="center", va="center", color=color, fontsize=fontsize, weight='bold')

plt.tight_layout(pad=3)
ax.set_ylabel('Vérité terrain', fontsize=18, weight='bold')
ax.set_xlabel('Labels prédits', fontsize=18, weight='bold')

plt.savefig("c2si-oral.pdf", bbox_inches='tight', pad_inches=0.1)

In [98]:
np.median(np.abs(mine)), np.mean(np.abs(mine)), np.std(np.abs(mine))

In [97]:
np.median(np.abs(mine)), np.mean(np.abs(hers)), np.std(np.abs(hers))

In [99]:
np.median(difference), np.mean(difference), np.std(difference)